In [2]:
import sys
import os

sys.path.append(os.path.dirname(os.getcwd()))

os.chdir("../")

# Number of files in train vs val

In [3]:
import torch
import math

In [4]:
samples_dir = "/vol/bitbucket/bet20/neural-lam/data/era5_uk/samples/train"
num_files = len(os.listdir(samples_dir))
total_days = 31 * 7 + 28 + 30 * 4 # Jan, Mar, May, Jul, Aug, Oct, Dec

total_time_steps = total_days * 4
pred_length = 1
sample_length = pred_length + 2
total_samples = total_time_steps - sample_length + 1
total_batches = math.ceil(total_samples / 4) 

print("Training:")
print("num_files:", num_files)
print("total_time_steps:", total_time_steps)
print("total_days:", total_days)
print("total_samples:", total_samples)
print("total_batches:", total_batches)

Training:
num_files: 1460
total_time_steps: 1460
total_days: 365
total_samples: 1458
total_batches: 365


In [5]:
# Finding the validation files 

samples_dir = "/vol/bitbucket/bet20/neural-lam/data/era5_uk/samples/val"
dirs = os.listdir(samples_dir)
num_files = len(dirs)
print(dirs)
print(num_files)

['01', '10', '04', 'other', '07']
5


In [6]:
# Verify number of validation samples
total_files = 0
for m in ['01', '04', '07', '10']:
    total_files += len(os.listdir(os.path.join(samples_dir, m)))
    

total_days = 31 + 30 + 31 + 31
total_time_steps = total_days * 4
pred_length = 28
sample_length = pred_length + 2
total_samples = total_time_steps - sample_length + 1
total_batches = math.ceil(total_samples / 4) 

print("Validation:")
print("total_files:", total_files)
print("total_time_steps:", total_time_steps)
print("total_days:", total_days)
print("total_samples:", total_samples)

Validation:
total_files: 492
total_time_steps: 492
total_days: 123
total_samples: 463


In [7]:
def steps(total_days):
    total_time_steps = total_days * 4
    pred_length = 28
    sample_length = pred_length + 2
    total_samples = total_time_steps - sample_length + 1
    return total_samples

days = [31, 30, 31, 31]
total_samples = sum([steps(d) for d in days])
total_batches = math.ceil(total_samples / 4)
print("total_batches:", total_batches)

total_batches: 94


In [8]:
import torch
from neural_lam.era5_dataset import ERA5UKDataset

In [9]:
dataset_name = "era5_uk"
batch_size = 4
n_workers = 4

train_set = ERA5UKDataset(
    dataset_name,
    pred_length=1,
    split="train",
    standardize=False,
)
train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

print("training_batches:", len(train_loader))

training_batches: 365


In [10]:
dataiter = iter(train_loader)
init_states, target_states, forcing = next(dataiter)
init_states

tensor([[[[ 1.9972e+05,  1.3039e+05,  9.8666e+04,  ...,  1.4843e-02,
            9.5106e-02, -2.3201e-02],
          [ 1.9972e+05,  1.3041e+05,  9.8663e+04,  ...,  1.9946e-02,
            6.3557e-02, -4.0367e-02],
          [ 1.9973e+05,  1.3043e+05,  9.8660e+04,  ...,  2.2730e-02,
            4.7319e-02, -2.8305e-02],
          ...,
          [ 1.9832e+05,  1.2941e+05,  9.6925e+04,  ...,  2.3893e-01,
           -4.6117e-01, -2.0646e-01],
          [ 1.9840e+05,  1.2945e+05,  9.6922e+04,  ...,  2.6259e-01,
           -1.7630e-01, -1.5450e-01],
          [ 1.9848e+05,  1.2949e+05,  9.6918e+04,  ...,  8.3043e-02,
            1.3593e-01, -4.6774e-04]],

         [[ 1.9984e+05,  1.3060e+05,  9.8931e+04,  ...,  1.4011e-01,
            1.2248e-01,  7.0517e-02],
          [ 1.9985e+05,  1.3063e+05,  9.8960e+04,  ...,  1.6331e-01,
            1.3269e-01,  6.3557e-02],
          [ 1.9986e+05,  1.3065e+05,  9.8989e+04,  ...,  1.8743e-01,
            1.3593e-01,  8.1651e-02],
          ...,
     

In [11]:
val_set = ERA5UKDataset(
    dataset_name,
    pred_length=28,
    split="val",
    standardize=False,
)
val_loader = torch.utils.data.DataLoader(
    val_set,
    batch_size,
    shuffle=True,
    num_workers=n_workers,
)

print("validation_set batches:", len(val_loader))

validation_set batches: 94


In [13]:
init_states, target_states, forcing = val_set[0]

In [14]:
print("init_states:", init_states.shape)
print("target_states:", target_states.shape)
print("forcing:", forcing.shape)

init_states: torch.Size([2, 3705, 48])
target_states: torch.Size([28, 3705, 48])
forcing: torch.Size([28, 3705, 0])
